In [32]:
import pandas as pd
import json

In [33]:
with open('trial_task.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

df = pd.json_normalize(data, record_path='products', meta=['order_id', 'warehouse_name', 'highway_cost'])

In [34]:
df

,product,price,quantity,order_id,warehouse_name,highway_cost
0,ломтик июльского неба,450,1,11973,Мордор,-70
1,билет в Израиль,1000,3,11973,Мордор,-70
2,статуэтка Ленина,200,3,11973,Мордор,-70
3,билет в Израиль,1000,1,62239,хутор близ Диканьки,-15
4,зеленая пластинка,10,2,85794,отель Лето,-50
...,...,...,...,...,...,...
192,автограф Стаса Барецкого,600,1,79293,отель Лето,-75
193,ломтик июльского неба,450,1,79293,отель Лето,-75
194,плюмбус,250,2,2930,Мордор,-30
195,плюмбус,250,1,2930,Мордор,-30


## Сперва я бы поменял отрицательные значения на положительные, в дальнейшем расходы, цена тарифа и тд будут иметь положительный знак

In [35]:
df['highway_cost'] = -1*df['highway_cost']

In [36]:
warehouse_quantities = df.groupby(['order_id', 'warehouse_name'], as_index=False).sum('quantity')

In [37]:
warehouse_quantities.rename(columns={"quantity": "sum_quantity"}, inplace=True)
warehouse_quantities.drop(columns=['price'], inplace=True)

In [38]:
warehouse_quantities

,order_id,warehouse_name,sum_quantity
0,124,хутор близ Диканьки,3
1,1391,остров невезения,2
2,2091,отель Лето,4
3,2108,остров невезения,2
4,2558,хутор близ Диканьки,5
...,...,...,...
95,98100,остров невезения,2
96,98423,хутор близ Диканьки,2
97,99220,хутор близ Диканьки,5
98,99246,хутор близ Диканьки,3


## на всякий случаи проверяем все расходы на доставку, чтобы не было отправки по разным ценам со склада

In [39]:
warehouse_costs = df[['order_id', 'warehouse_name', 'highway_cost']].drop_duplicates(ignore_index=True)

In [40]:
warehouse_subscription_prices = warehouse_costs.merge(warehouse_quantities, how='left', on=['order_id', 'warehouse_name'])

In [41]:
warehouse_subscription_prices['subscription_price'] = warehouse_subscription_prices['highway_cost']/warehouse_subscription_prices['sum_quantity']

In [42]:
warehouse_subscription_prices

,order_id,warehouse_name,highway_cost,sum_quantity,subscription_price
0,11973,Мордор,70,7,10.0
1,62239,хутор близ Диканьки,15,1,15.0
2,85794,отель Лето,50,2,25.0
3,33684,Мордор,30,3,10.0
4,25824,отель Лето,75,3,25.0
...,...,...,...,...,...
95,28106,хутор близ Диканьки,30,2,15.0
96,98100,остров невезения,10,2,5.0
97,79293,отель Лето,75,3,25.0
98,2930,Мордор,30,3,10.0


In [43]:
for warehouse in warehouse_subscription_prices['warehouse_name'].unique():
  temp = warehouse_subscription_prices[warehouse_subscription_prices['warehouse_name']==warehouse]
  print(temp['subscription_price'].max() - temp['subscription_price'].min())

0.0
0.0
0.0
0.0
0.0


In [44]:
warehouse_subscription_prices['warehouse_name'].unique()

array(['Мордор', 'хутор близ Диканьки', 'отель Лето', 'остров невезения',
       'гиперборея'], dtype=object)

In [45]:
warehouse_subscription_prices['warehouse_name'].nunique()

5

In [46]:
warehouse_subscription_prices['subscription_price'].unique()

array([10.0, 15.0, 25.0, 5.0, 20.0], dtype=object)

In [47]:
warehouse_subscription_prices['subscription_price'].nunique()

5

### видим что везде товары отправились по одним и тем же ценам со склада

# ответ на первую задачу:
Найти тариф стоимости доставки для каждого склада

In [53]:
warehouse_subscription_prices = warehouse_subscription_prices.groupby('warehouse_name', as_index=False).first()[['warehouse_name', 'subscription_price']]

In [55]:
warehouse_subscription_prices

,warehouse_name,subscription_price
0,Мордор,10.0
1,гиперборея,20.0
2,остров невезения,5.0
3,отель Лето,25.0
4,хутор близ Диканьки,15.0


Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами
'product', 'quantity', 'income', 'expenses', 'profit')

In [50]:
df['total_sale_value'] = df['price'] * df['quantity']

In [51]:
income_quantity_df = df.groupby('product', as_index=False).sum('quantity')[['product', 'quantity', 'total_sale_value']]

In [57]:
income_quantity_df

,product,quantity,total_sale_value
0,автограф Стаса Барецкого,48,28800
1,билет в Израиль,58,58000
2,зеленая пластинка,61,610
3,ломтик июльского неба,42,18900
4,плюмбус,65,16250
5,подписка на suppi-блог,33,4950
6,статуэтка Ленина,68,13600


In [58]:
df_with_subscription = df.merge(warehouse_subscription_prices, how='left', on='warehouse_name')

In [59]:
df_with_subscription['expenses'] = df_with_subscription['quantity']*df_with_subscription['subscription_price']

In [62]:
expenses_df = df_with_subscription.groupby('product', as_index=False).sum('expenses')[['product', 'expenses']]

In [64]:
task_2 = income_quantity_df.merge(expenses_df, how='left', on='product')

In [65]:
task_2.rename(columns={'total_sale_value': 'income'}, inplace=True)

In [68]:
task_2['profit'] = task_2['income'] - task_2['expenses']

# ответ на вторую задачу:

In [69]:
task_2

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,820.0,27980.0
1,билет в Израиль,58,58000,1050.0,56950.0
2,зеленая пластинка,61,610,920.0,-310.0
3,ломтик июльского неба,42,18900,670.0,18230.0
4,плюмбус,65,16250,940.0,15310.0
5,подписка на suppi-блог,33,4950,545.0,4405.0
6,статуэтка Ленина,68,13600,935.0,12665.0


### ps. Я бы посоветовал продавать товар 'зеленая пластинка' только со склада 'остров невезения', так как стоимость доставки стоит дешевле чем цена товара только в этом складе.

In [75]:
df[df['order_id'].isin(df[df['product'] == 'зеленая пластинка']['order_id'])]

,product,price,quantity,order_id,warehouse_name,highway_cost,total_sale_value
4,зеленая пластинка,10,2,85794,отель Лето,50,20
5,билет в Израиль,1000,2,33684,Мордор,30,2000
6,зеленая пластинка,10,1,33684,Мордор,30,10
17,зеленая пластинка,10,2,96473,Мордор,20,20
18,плюмбус,250,2,64013,хутор близ Диканьки,105,500
...,...,...,...,...,...,...,...
180,плюмбус,250,1,15237,хутор близ Диканьки,60,250
181,зеленая пластинка,10,2,15237,хутор близ Диканьки,60,20
183,зеленая пластинка,10,1,70849,остров невезения,25,10
184,плюмбус,250,2,70849,остров невезения,25,500


In [78]:
df[(df['warehouse_name']=='остров невезения') & (df['product']=='зеленая пластинка')]

,product,price,quantity,order_id,warehouse_name,highway_cost,total_sale_value
52,зеленая пластинка,10,2,59930,остров невезения,20,20
82,зеленая пластинка,10,3,72097,остров невезения,30,30
110,зеленая пластинка,10,1,2108,остров невезения,10,10
183,зеленая пластинка,10,1,70849,остров невезения,25,10
185,зеленая пластинка,10,2,70849,остров невезения,25,20


Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

In [85]:
order_income = df.groupby('order_id', as_index=False).sum('total_sale_value')[['order_id', 'total_sale_value']]

In [92]:
order_expenses = df.groupby('order_id', as_index=False).first()[['order_id', 'highway_cost']]

In [94]:
order_sales = order_income.merge(order_expenses, how='left', on='order_id')

In [98]:
order_sales.rename(columns={"total_sale_value":"income", "highway_cost":"expenses"}, inplace=True)

In [102]:
order_sales['profit'] = order_sales['income'] - order_sales['expenses']

# ответ на третье задание:

In [104]:
order_sales[['order_id', 'profit']]

,order_id,profit
0,124,705
1,1391,490
2,2091,1300
3,2108,200
4,2558,355
...,...,...
95,98100,1590
96,98423,1170
97,99220,1075
98,99246,1755


видим что некоторые продажи вышли нам в минус

In [111]:
order_sales[order_sales['profit']<0]

,order_id,income,expenses,profit
5,2751,40,80,-40
72,76803,10,25,-15
86,85794,20,50,-30


In [110]:
df[df['order_id'].isin(order_sales[order_sales['profit']<0]['order_id'])]

,product,price,quantity,order_id,warehouse_name,highway_cost,total_sale_value
4,зеленая пластинка,10,2,85794,отель Лето,50,20
26,зеленая пластинка,10,3,2751,гиперборея,80,30
27,зеленая пластинка,10,1,2751,гиперборея,80,10
161,зеленая пластинка,10,1,76803,отель Лето,25,10


Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

In [132]:
warehouse_income = df.groupby('warehouse_name', as_index=False).sum('total_sale_value')[['warehouse_name', 'total_sale_value']]

In [134]:
warehouse_income.rename(columns={"total_sale_value":"income"}, inplace=True)

In [131]:
warehouse_expenses = df.groupby(['warehouse_name', 'order_id'], as_index=False).first().groupby('warehouse_name', as_index=False).sum('highway_cost')[['warehouse_name', 'highway_cost']]

In [136]:
warehouse_expenses.rename(columns={"highway_cost":"expenses"}, inplace=True)

In [138]:
warehouse_sales = warehouse_income.merge(warehouse_expenses, how='left', on='warehouse_name')

In [161]:
warehouse_sales['profit_for_warehouse'] = warehouse_sales['income'] - warehouse_sales['expenses']

In [164]:
warehouse_sales

,warehouse_name,income,expenses,profit_for_warehouse
0,Мордор,17560,510,17050
1,гиперборея,40000,1720,38280
2,остров невезения,15890,280,15610
3,отель Лето,28650,1600,27050
4,хутор близ Диканьки,39010,1770,37240


In [145]:
warehouse_product_income = df.groupby(['warehouse_name', 'product'], as_index=False).sum('total_sale_value')[['warehouse_name', 'product', 'total_sale_value']]

In [151]:
warehouse_product_expenses = df_with_subscription.groupby(['warehouse_name', 'product'], as_index=False).sum('expenses')[['warehouse_name', 'product', 'expenses']]

In [152]:
warehouse_product_sales = warehouse_product_income.merge(warehouse_product_expenses, how='left', on=['warehouse_name', 'product'])

In [154]:
warehouse_product_sales.rename(columns={"total_sale_value":"income"}, inplace=True)

In [157]:
warehouse_product_sales['profit_for_product_in_warehouse'] = warehouse_product_sales['income'] - warehouse_product_sales['expenses']

In [168]:
warehouse_product = warehouse_product_sales.merge(warehouse_sales, how='left', on='warehouse_name')[['warehouse_name', 'product', 'profit_for_product_in_warehouse', 'profit_for_warehouse']]

In [171]:
warehouse_product['percent_profit_product_of_warehouse'] = 100*warehouse_product['profit_for_product_in_warehouse'] / warehouse_product['profit_for_warehouse']

# ответ на четвертое задание

In [172]:
warehouse_product

,warehouse_name,product,profit_for_product_in_warehouse,profit_for_warehouse,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,2360.0,17050,13.841642
1,Мордор,билет в Израиль,8910.0,17050,52.258065
2,Мордор,зеленая пластинка,0.0,17050,0.000000
3,Мордор,ломтик июльского неба,1320.0,17050,7.741935
4,Мордор,плюмбус,1440.0,17050,8.445748
5,Мордор,подписка на suppi-блог,1120.0,17050,6.568915
6,Мордор,статуэтка Ленина,1900.0,17050,11.143695
7,гиперборея,автограф Стаса Барецкого,6960.0,38280,18.181818
8,гиперборея,билет в Израиль,20580.0,38280,53.761755
9,гиперборея,зеленая пластинка,-100.0,38280,-0.261233


Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться
'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [179]:
warehouse_product = warehouse_product.sort_values(by='percent_profit_product_of_warehouse', ascending=False, ignore_index=True)

In [197]:
warehouse_product['accumulated_percent_profit_product_of_warehouse'] = warehouse_product['percent_profit_product_of_warehouse'].iloc[::1].cumsum()

# ответ на пятое задание

In [200]:
warehouse_product

,warehouse_name,product,profit_for_product_in_warehouse,profit_for_warehouse,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,отель Лето,билет в Израиль,14625.0,27050,54.066543,54.066543
1,гиперборея,билет в Израиль,20580.0,38280,53.761755,107.828299
2,Мордор,билет в Израиль,8910.0,17050,52.258065,160.086363
3,хутор близ Диканьки,автограф Стаса Барецкого,12870.0,37240,34.559613,194.645977
4,остров невезения,ломтик июльского неба,4450.0,15610,28.507367,223.153344
5,хутор близ Диканьки,билет в Израиль,9850.0,37240,26.450054,249.603398
6,остров невезения,статуэтка Ленина,3510.0,15610,22.485586,272.088984
7,остров невезения,плюмбус,3430.0,15610,21.973094,294.062078
8,остров невезения,билет в Израиль,2985.0,15610,19.122357,313.184435
9,гиперборея,автограф Стаса Барецкого,6960.0,38280,18.181818,331.366254


Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A.
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

In [203]:
warehouse_product['category'] = warehouse_product['accumulated_percent_profit_product_of_warehouse'].apply(lambda x: 'A' if x <= 70 else ('B' if x <= 90 else 'C'))

# ответ на шестое задание

In [204]:
warehouse_product

,warehouse_name,product,profit_for_product_in_warehouse,profit_for_warehouse,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,отель Лето,билет в Израиль,14625.0,27050,54.066543,54.066543,A
1,гиперборея,билет в Израиль,20580.0,38280,53.761755,107.828299,C
2,Мордор,билет в Израиль,8910.0,17050,52.258065,160.086363,C
3,хутор близ Диканьки,автограф Стаса Барецкого,12870.0,37240,34.559613,194.645977,C
4,остров невезения,ломтик июльского неба,4450.0,15610,28.507367,223.153344,C
5,хутор близ Диканьки,билет в Израиль,9850.0,37240,26.450054,249.603398,C
6,остров невезения,статуэтка Ленина,3510.0,15610,22.485586,272.088984,C
7,остров невезения,плюмбус,3430.0,15610,21.973094,294.062078,C
8,остров невезения,билет в Израиль,2985.0,15610,19.122357,313.184435,C
9,гиперборея,автограф Стаса Барецкого,6960.0,38280,18.181818,331.366254,C
